In [12]:
import re
import math
import numpy as np
from PIL import Image

try:
    # Unos fotografije koja se obradjuje
    # S obzirom da su pikseli u fotografiji predstavljeni matricama,
    # potrebno je enkodovati brojeve u njima

    unos = int(input("Unesite 1 ukoliko unosite RGB fajl, a 2 ukoliko unosite crno-beli fajl:"))

    if unos == 1:
        file = input("Ime fajla:")
        my_string = np.asarray(Image.open(file), np.uint8)
        shape = my_string.shape
        a = my_string
        my_string = str(my_string.tolist())
    elif unos == 2:
        array = np.arange(0, 737280, 1, np.uint8)
        my_string = np.reshape(array, (1024, 720))
        a = my_string
        my_string = str(my_string.tolist())
    else:
        raise ValueError("Uneli ste nepodržan unos")

except FileNotFoundError:
    print("Fajl ne postoji. Bacam izuzetak.")

except ValueError as ve:
    print(ve)








Unesite 1 ukoliko unosite RGB fajl, a 2 ukoliko unosite crno-beli fajl:1
Ime fajla:cat.jpg


In [16]:
# racunanje frekvencije pojavljivanja odredjenih brojeva koji reprezentuju
# boje na fotografiji i sortiranje

letters = []
only_letters = []
for letter in my_string:
    if letter not in letters:
        frequency = my_string.count(letter)
        letters.append(frequency)
        letters.append(letter)
        only_letters.append(letter)

nodes = []
while len(letters) > 0:
    nodes.append(letters[0:2])
    letters = letters[2:]
nodes.sort()
huffman_tree = []
huffman_tree.append(nodes)

# kreiranje Hafmenovog stabla, i postavljanje jedinstvenih vrednosti
# kao listove stabla

def combine_nodes(nodes):
    pos = 0
    newnode = []
    if len(nodes) > 1:
        nodes.sort()
        nodes[pos].append("1")
        nodes[pos+1].append("0")
        combined_node1 = (nodes[pos] [0] + nodes[pos+1] [0])
        combined_node2 = (nodes[pos] [1] + nodes[pos+1] [1])
        newnode.append(combined_node1)
        newnode.append(combined_node2)
        newnodes=[]
        newnodes.append(newnode)
        newnodes = newnodes + nodes[2:]
        nodes = newnodes
        huffman_tree.append(nodes)
        combine_nodes(nodes)
    return huffman_tree

newnodes = combine_nodes(nodes)

huffman_tree.sort(reverse = True)

checklist = []
for level in huffman_tree:
    for node in level:
        if node not in checklist:
            checklist.append(node)
        else:
            level.remove(node)


In [19]:
# generisanje binarnog koda
count = 0

letter_binary = []
if len(only_letters) == 1:
    lettercode = [only_letters[0], "0"]
    letter_binary.append(lettercode*len(my_string))
else:
    for letter in only_letters:
        code =""
        for node in checklist:
            if len (node)>2 and letter in node[1]:
                code = code + node[2]
        lettercode =[letter,code]
        letter_binary.append(lettercode)

print("Generisani binarni kod:")
for letter in letter_binary:
    print(letter[0], letter[1])

bitstring =""
for character in my_string:
    for item in letter_binary:
        if character in item:
            bitstring = bitstring + item[1]
binary ="0b"+bitstring


uncompressed_file_size = len(my_string)*7
compressed_file_size = len(binary)-2
print("Originalna veličina fajla bila je ", uncompressed_file_size," bita. Komprimovana veličina je: ",compressed_file_size)
print("Ovime smo uštedeli prostor od ",uncompressed_file_size-compressed_file_size,"bita")
output = open("compressed.txt","w+")
print("Komprimovani fajl je generisan kao compressed.txt")
output = open("compressed.txt","w+")
output.write(bitstring)
efikasnost = (uncompressed_file_size - compressed_file_size) / uncompressed_file_size *100
print("Postignuta efikasnost iznosi", efikasnost, " %.")

def entropija(letter_binary):
    entropy = 0
    for letter_code in letter_binary:
        probability = my_string.count(letter_code[0]) / len(my_string)
        entropy -= probability * np.log2(probability)
    return entropy

entropija = entropija(letter_binary)
print("Entropija je: ", entropija)


Generisani binarni kod:
[ 0111
2 0010
4 1010
9 01010
, 11
  000
3 1011
] 0110
7 00111
1 100
0 01011
5 00110
8 01000
6 01001
Originalna veličina fajla bila je  41507130  bita. Komprimovana veličina je:  21009169
Ovime smo uštedeli prostor od  20497961 bita
Komprimovani fajl je generisan kao compressed.txt
Postignuta efikasnost iznosi 49.38419254715997  %.
Entropija je:  3.494543312095562


In [ ]:
# dekodovanje fotografije

bitstring = str(binary[2:])
uncompressed_string =""
code =""
for digit in bitstring:
    code = code+digit
    pos=0                                        #iterating and decoding
    for letter in letter_binary:
        if code ==letter[1]:
            uncompressed_string=uncompressed_string+letter_binary[pos] [0]
            code=""
        pos+=1

if unos == 1:
    temp = re.findall(r'\d+', uncompressed_string)
    res = list(map(int, temp))
    res = np.array(res)
    res = res.astype(np.uint8)
    res = np.reshape(res, shape)
    #print(res)
    print("Posmatrajte da li se ulazne dimenzije razlikuju od izlaznih.")
    print("Dimenzije ulazne fotografije:",shape)
    print("Dimenzije izlazne fotografije:",res.shape)
    data = Image.fromarray(res)
    data.save('komprimovano.png')
    if a.all() == res.all():
        print("Kompresija je uspesno izvrsena!")
if unos == 2:
    temp = re.findall(r'\d+', uncompressed_string)
    res = list(map(int, temp))
    res = np.array(res)
    res = res.astype(np.uint8)
    res = np.reshape(res, (1024, 720))
    data = Image.fromarray(res)
    data.save('komprimovano.png')
    print("Kompresija je uspesno izvrsena!")

